# ONNX Export Demo

Reference: https://pytorch.org/tutorials//beginner/onnx/export_simple_model_to_onnx_tutorial.html#save-the-onnx-model-in-a-file

## Train a simple linear regression model

In [7]:
%pip install -q tensorweaver netron==8.3.5

Note: you may need to restart the kernel to use updated packages.


In [8]:
import tensorweaver as torch # OR import torch
from tensorweaver.toy_datasets.get_celsius_fahrenheit_dataset import (
    get_celsius_fahrenheit_dataset,
)

In [9]:
# load the dataset
x_array, y_array = get_celsius_fahrenheit_dataset()

In [10]:
# define our model, a simple linear regression model
class LinearRegressionModel(torch.nn.Module):

    def __init__(self):
        super(LinearRegressionModel, self).__init__()
        self.linear = torch.nn.Linear(1, 1)  # One in and one out

    def forward(self, x):
        y_pred = self.linear(x)
        return y_pred


# create our model
our_model = LinearRegressionModel()

# convert the dataset to tensors
x = torch.tensor(x_array)  # temperature in celsius
y = torch.tensor(y_array)  # temperature in fahrenheit

# define our loss function and optimizer
criterion = torch.nn.MSELoss()
optimizer = torch.optim.SGD(our_model.parameters(), lr=0.0015)

# train our model
loss_path = []

for epoch in range(10000):
    # Forward pass: Compute predicted y by passing
    # x to the model
    pred_y = our_model(x)

    # Compute and print loss
    loss = criterion(pred_y, y)

    # Zero gradients, perform a backward pass,
    # and update the weights.
    optimizer.zero_grad()
    loss_path.append(loss.item())
    loss.backward()
    optimizer.step()

    if epoch % 1000 == 0:
        print('epoch {}, loss {}'.format(epoch, loss.item()))

epoch 0, loss 2234.7233454369148
epoch 1000, loss 6.180777208148875
epoch 2000, loss 0.848490979961047
epoch 3000, loss 0.8151145690666656
epoch 4000, loss 0.8149056558994958
epoch 5000, loss 0.8149043482480652
epoch 6000, loss 0.8149043400630761
epoch 7000, loss 0.8149043400118421
epoch 8000, loss 0.8149043400115227
epoch 9000, loss 0.8149043400115212


## Export our trained model to ONNX format

In [11]:
example_inputs = (torch.tensor(((0.0,), (100.0,))),)
onnx_program = torch.onnx.export(our_model, example_inputs, "model.onnx")

## Display our exported ONNX model by using netron

In [12]:
import netron
address = netron.serve('model.onnx')
netron.widget(address, height=800)

Serving 'model.onnx' at http://localhost:8080


## Using onnxruntime to load and run our exported ONNX model

In [13]:
%pip install -q onnxruntime

import onnxruntime

onnx_inputs = [tensor.numpy() for tensor in example_inputs]
print(f"Sample input: {onnx_inputs}")

 # will solve wired errors on some machines: pthread_setaffinity_np
session_options = onnxruntime.SessionOptions()
session_options.intra_op_num_threads = 1

ort_session = onnxruntime.InferenceSession(
    "./model.onnx",
    providers=["CPUExecutionProvider"],
    sess_options=session_options
)

onnxruntime_input = {input_arg.name: input_value for input_arg, input_value in zip(ort_session.get_inputs(), onnx_inputs)}

# ONNX Runtime returns a list of outputs
onnxruntime_outputs = ort_session.run(None, onnxruntime_input)[0]

print("onnx runtime outputs: ", onnxruntime_outputs)
print("expected outputs: 0 celsius -> 32 fahrenheit, 100 celsius -> 212 fahrenheit")

Note: you may need to restart the kernel to use updated packages.
Sample input: [array([[  0.],
       [100.]])]
onnx runtime outputs:  [[ 31.87891188]
 [212.05132755]]
expected outputs: 0 celsius -> 32 fahrenheit, 100 celsius -> 212 fahrenheit
